In [1]:
import cv2
import numpy as np
import os
import matplotlib as plt

In [2]:
#KNN code
def distance(v1,v2):
    return np.sqrt(((v1-v2)**2).sum())


def KNN(train,test,k=5):
    dist=[]
    
    for i in range(train.shape[0]):
        #get the vector and label
        ix=train[i, :-1]
        iy=train[i, -1]
        #compute the distance form the test point
        d=distance(test,ix)
        dist.append([d,iy])
        
    #sort based on distance and get top k
    dk=sorted(dist,key=lambda x:x[0])[:k]
    #retrieve only labels
    labels=np.array(dk)[:,-1]
    
    
    #get frequence of each label
    output=np.unique(labels,return_counts=True)
    #find max frequency and corresponding label
    index=np.argmax(output[1])
    return output[0][index]

In [13]:
#init the carema
cap=cv2.VideoCapture(0)

In [3]:
#face detection
face_cascade=cv2.CascadeClassifier("/Users/anuragsingh/Desktop/cdMLAI/HaarCascade/haarcascade_frontalface_alt.xml")

In [4]:
skip=0
dataset_path='/Users/anuragsingh/Desktop/cdMLAI/Data/Face/'
face_data=[]
label=[]

class_id=0#labels for the given file
name={}#mapping btw id and name

In [5]:

#Data Preparation


In [6]:
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        #Create a mapping btw class_id and name
        name[class_id] = fx[:-4]
        print("Loaded "+fx)
        data_item = np.load(dataset_path+fx)
        face_data.append(data_item)
        
        
        #create labels for the class
        print(data_item.shape)
        target=class_id*np.ones((data_item.shape[0],))
        class_id+=1
        label.append(target)

Loaded satrudhan.npy
(21, 30000)
Loaded Anurag.npy
(33, 30000)


In [7]:
face_dataset=np.concatenate(face_data,axis=0)
face_labels=np.concatenate(label,axis=0).reshape((-1,1))
print(face_dataset.shape)
print(face_labels.shape)


trainset=np.concatenate((face_dataset,face_labels),axis=1)
print(trainset.shape)#it will have one extra col for labels



(54, 30000)
(54, 1)
(54, 30001)


In [8]:

#testing


In [ ]:
while True:
    ret,frame=cap.read()
    
    if ret==False:
        continue
        
    faces=face_cascade.detectMultiScale(frame,1.3,5)
    
    for face in faces:
        x,y,w,h=face
        
        #get the face region of Interest
        offset=10
        face_section=frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section=cv2.resize(face_section,(100,100))#resize the shape of the image
        
        
        
        
        #predicted label
        out=KNN(trainset,face_section.flatten())
        
        
        
        
        
        #display on the screen the name and rectangle around it
        pred_name=name[int(out)]
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        
        
    cv2.imshow("Faces",frame)
    
    key=cv2.waitKey(1) & 0xFF
    if key==ord('q'):
        break

        
        
cap.release()
cv2.destroyAllWindows()

In [12]:
os.listdir(dataset_path)
#print(dataset_path)

['satrudhan.npy', 'Anurag.npy']